In [1]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD, SSN, SOSA #most common namespaces
import urllib.parse #for parsing strings to URI's
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
# import stardog

In [2]:
df = pd.read_csv('Jasmin.csv')

In [3]:
df['Altitude']= pd.to_numeric(df['Altitude'],errors='coerce')
df['Altitude'].mean()

143.75714285714284

In [4]:
df.head(10)

,ID,LocalDate,LocalTime,GMTDate,GMTTime,lat,long,Temperature,Speed,Direction,Altitude,Cov,HDOP,Distance,Count
0,1SAT32,2011-10-26,07:40:35,2011-10-25,23:40:35,2.437206,185.111301,24.5,0.24,0,NaN,0,1.64,0,1
1,2SAT32,2011-10-26,15:41:35,2011-10-26,7:41:35,2.433402,185.108162,29.0,0.76,0,NaN,0,1.74,548,2
2,3SAT32,2011-10-26,23:40:35,2011-10-26,15:40:35,2.433734,185.108599,27.0,0.86,0,NaN,0,2.18,61,3
3,4SAT32,2011-10-27,23:40:34,2011-10-27,15:40:34,2.455738,185.183640,28.0,0.16,0,NaN,0,1.89,8669,4
4,5SAT32,2011-10-28,07:42:53,2011-10-27,23:42:53,2.456503,185.187684,27.5,1.46,0,NaN,1,2.50,456,5
5,6SAT32,2011-10-28,15:41:05,2011-10-28,7:41:05,2.483344,185.232541,33.5,0.06,0,NaN,5,1.45,5800,6
6,7SAT32,2011-10-28,15:41:05,2011-10-28,7:41:05,2.483344,185.232541,33.5,0.06,0,NaN,5,1.45,0,7
7,8SAT32,2011-10-28,15:41:05,2011-10-28,7:41:05,2.483344,185.232541,33.5,0.06,0,NaN,5,1.45,0,8
8,9SAT32,2011-10-28,19:40:34,2011-10-28,11:40:34,2.479701,185.243606,29.0,0.14,0,NaN,4,0.95,1291,9
9,10SAT32,2011-10-28,19:40:34,2011-10-28,11:40:34,2.479701,185.243606,29.0,0.14,0,NaN,5,0.95,0,10


In [5]:
df.describe()

,lat,long,Temperature,Speed,Direction,Altitude,Cov,HDOP,Distance,Count
count,899.000000,899.000000,899.000000,899.000000,899.000000,7.000000,899.000000,899.000000,899.000000,899.000000
mean,2.509015,183.746772,28.888765,0.587164,1.329255,143.757143,2.497219,2.550044,640.407119,450.000000
std,0.073845,9.925183,4.016236,0.673300,17.429289,88.443143,2.170552,3.812480,917.534269,259.663243
min,2.392483,117.981100,-35.500000,0.000000,0.000000,101.500000,0.000000,0.710000,0.000000,1.000000
25%,2.465016,185.199810,28.000000,0.180000,0.000000,103.050000,0.000000,1.295000,145.500000,225.500000
50%,2.490733,185.248029,28.500000,0.370000,0.000000,114.900000,2.000000,1.830000,341.000000,450.000000
75%,2.546202,185.311595,30.000000,0.735000,0.000000,120.200000,5.000000,2.715000,761.000000,674.500000
max,2.706847,185.403456,37.500000,6.610000,343.000000,343.400000,5.000000,99.900000,9603.000000,899.000000


In [6]:
g = Graph()
ID = Namespace('foo_')
owl = Namespace('http://www.w3.org/2002/07/owl#')
sosa = Namespace('http://www.w3.org/ns/sosa/')
foo = Namespace('http://www.ontology/ns/foo/1.1#')
lat = Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
long =Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
alt = Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
time = Namespace('http://www.w3.org/2006/time#')
schema = Namespace('http://schema.org/')
XSD=Namespace('http://www.w3.org/2001/XMLSchema#')
Class = Namespace('http://purl.org/ontology/wo/Class/')
Family = Namespace('http://purl.org/ontology/wo/Family/')
Kingdom = Namespace('http://purl.org/ontology/wo/kingdom/')
Order = Namespace('http://purl.org/ontology/wo/Proboscidea/')
Phylum = Namespace('http://purl.org/ontology/wo/Phylum/')

In [8]:
for index, row in df.iterrows():
    g.add((URIRef(foo+row['ID']), RDF.type, SOSA.Observation))
    g.add((URIRef(ID+row['ID']), foo.ID, Literal(row['ID'], datatype=XSD.string))) 

    g.add((URIRef(schema+'DGFC/elephant#Jasmin'), RDF.type,  sosa.Sensor))
    g.add((URIRef(ID+row['ID']), URIRef(schema+'dgfc/elephant#Jasmin'), Literal(row['ID'], datatype=XSD.string))) 
          
    g.add((URIRef(foo+'LocalDate'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), time.LocalDate, Literal(row['LocalDate'], datatype=XSD.date)))
    
    g.add((URIRef(foo+'LocalTime'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']),  time.LocalTime, Literal(row['LocalTime'], datatype=XSD.time)))
      
    g.add((URIRef(foo+'GMTDate'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), time.GMTDate, Literal(row['GMTDate'], datatype=XSD.date)))

    g.add((URIRef(foo+'GMTTime'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), time.GMTTime,  Literal(row['GMTTime'], datatype=XSD.time)))

    g.add((URIRef(lat+'lat'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), lat.lat, Literal(row['lat'], datatype=XSD.float)))
     
    g.add((URIRef(long+'long'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), long.long, Literal(row['long'], datatype=XSD.float)))
    
    g.add((URIRef(foo+'Temperature'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Temperature, Literal(row['Temperature'], datatype=XSD.double)))
    
    g.add((URIRef(foo+'Speed'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Speed, Literal(row['Speed'], datatype=XSD.float)))
    
    g.add((URIRef(alt+'alt'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']),  alt.alt, Literal(row['Altitude'], datatype=XSD.float)))
   
    g.add((URIRef(foo+'Direction'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Direction, Literal(row['Direction'], datatype=XSD.float)))
    
    g.add((URIRef(foo+'Distance'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Distance,  Literal(row['Distance'], datatype=XSD.float)))
    
    g.add((URIRef(foo+'HDOP'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.HDOP, Literal(row['HDOP'], datatype=XSD.integer)))
 
    g.add((URIRef(foo+'Cov'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Cov, Literal(row['Cov'], datatype=XSD.integer)))
    
    g.add((URIRef(foo+'Count'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Count, Literal(row['Count'], datatype=XSD.integer)))
    
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Class.Mammalia))
    g.add((URIRef(ID+row['ID']), URIRef(foo+'ElephasMaximus'), Literal(row['ID'], datatype=XSD.string)))
    
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Family.Elephantidae))
    g.add((URIRef(ID+row['ID']), URIRef(foo+'ElephasMaximus'), Literal(row['ID'], datatype=XSD.string))) 
    
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Kingdom.Animalia))
    g.add((URIRef(ID+row['ID']), URIRef(foo+'ElephasMaximus'), Literal(row['ID'], datatype=XSD.string))) 
     
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Phylum.Chordata))
    g.add((URIRef(ID+row['ID']), URIRef(foo+'ElephasMaximus'), Literal(row['ID'], datatype=XSD.string)))
    

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '7:41:35'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '3:41:06'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '3:40:35'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '7:41:37'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '7:40:34'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

...

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '7:40:34'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '7:40:34'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '7:41:07'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '7:41:04'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\isotime.py", line 148, in parse_time
    raise ISO8601Error('Unrecognised ISO 8601 time format: %r' % timestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 time format: '7:42:24'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#time, Converter=<function parse_time at 0x00000242D1820940>
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "C:\Users\admin\anaconda3\lib\site-packages\isodate\is

In [9]:
print(g.serialize(format='turtle')).head(1)

@prefix ns1: <http://www.ontology/ns/foo/1.1#> .
@prefix ns2: <http://www.w3.org/2006/time#> .
@prefix ns3: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ns4: <http://schema.org/dgfc/elephant#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<foo_100SAT32> ns4:Jasmin "100SAT32"^^xsd:string ;
    ns1:Count 100 ;
    ns1:Cov 4 ;
    ns1:Direction "0"^^xsd:float ;
    ns1:Distance "299"^^xsd:float ;
    ns1:ElephasMaximus "100SAT32"^^xsd:string ;
    ns1:HDOP 0.91 ;
    ns1:ID "100SAT32"^^xsd:string ;
    ns1:Speed "0.24"^^xsd:float ;
    ns1:Temperature 3.3e+01 ;
    ns3:alt "NaN"^^xsd:float ;
    ns3:lat "2.676665"^^xsd:float ;
    ns3:long "185.393313"^^xsd:float ;
    ns2:GMTDate "2011-11-13"^^xsd:date ;
    ns2:GMTTime "7:40:34"^^xsd:time ;
    ns2:LocalDate "2011-11-13"^^xsd:date ;
    ns2:LocalTime "15:40:34"^^xsd:time .

<foo_101SAT32> ns4:Jasmin "101SAT32"^^xsd:string ;
    ns1:Count 101 ;
    ns1:Cov 5 ;
    ns1:Direction "0"^^xsd:float ;
    ns1:Distance "430"^^xsd

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
# saving ontology to disk
g.serialize("Jasmin10march.rdf", format="ttl")